- http://www.universalteacherpublications.com/univ/ebooks/or/index1.htm
- https://katex.org/docs/supported.html


# Queuing Theory (Waiting Line Models)

http://www.universalteacherpublications.com/univ/ebooks/or/Ch10/mmcex.htm


## M/M/c Queuing System (∞/FIFO)

It is a queuing model where the arrivals follow a Poisson process, service times are exponentially distributed and there are c servers. In other words, it is a system with Poisson input, exponential waiting time and Poisson output.

Queue capacity of the system is infinite with first in first out mode. The first M in the notation stands for Poisson input (= exponential distribution of interarrival times), second M for Poisson output (= exponential distribution of processing times), c for the number of servers and ∞ for infinite capacity of the system.

### Formulas

 performance indicator                                    | Formula 
:---------------------------------------------------------|:--------
 utilization                                              | $\rho = \cfrac{\lambda}{c\mu}$
 Probability of zero unit in the queue ($P_o$)            | $1 - \cfrac{\lambda}{\mu}$
 Average queue length ($L_q$ )                            | $\cfrac{\lambda^2}{\mu(\mu-\lambda)}$
 Average number of units in the system ($L_s$)            | $\cfrac{\lambda}{\mu-\lambda}$
 Average waiting time of an arrival ($W_q$)               | $\cfrac{\lambda}{\mu(\mu-\lambda)}$
 Average waiting time of an arrival in the system ($W_s$) | $\cfrac{1}{\mu-\lambda}$

## Simulation code

Below a function is created to run the simulation with parameters, e.g.
- lambda, being the mean arrivals in cars/hour as poisson arrival distribution.
- mu, being the mean charging capacity in cars/hour as expontential distribution.
- n, being the number of charging stations

### Example 1

Cars arrive at a single charging station according to a Poisson input process with a mean rate of 40 per hour. The time required to charge a car has an exponential distribution with a mean of 50 per hour. Assume that the cars are charged by a single charging station, find the average waiting time of a car.

#### Solution

Given:

$\lambda = 40/hour$, $\mu = 50/hour$

Average waiting time of a car before receiving service ($W_q$) = $\cfrac{40}{50(50-40)}$ = 4.8 minutes


#### Simulation of Example 1

##### passivate/activate method

- [A bank example, with multiple clerks](https://www.salabim.org/manual/Modelling.html#a-bank-example)
- [sample models/Bank, 3 clerks.py (github)](https://github.com/salabim/salabim/blob/master/sample%20models/Bank%2C%203%20clerks.py)

###### Components:

- **CarGenerator:** generates cars with an exponential *inter arrival time* with a mean of 60/40 minutes resuting in 40 cars per hour according to a Possion distribution.
- **Car:** The car enters the waiting line and checks if any of the stations is available. 
In case it is the station is activated. 
- **ChargingStation:** Take the first car from the waiting line (FIFO), charge the car and activate the car as soon as the car is fully charged.  
- **Queue:** A waiting line for the cars waiting to be charged.

Statistics of Length of stay in Waiting Cars at     50000    
                        all    excl.zero         zero
-------------- ------------ ------------ ------------
entries           33265        26529         6736    
mean                  4.578        5.741
std.deviation         5.515        5.609

minimum               0            0.000
maximum              38.790       38.790


### Example 2

New Delhi Charging Hub has a single charging station. During the rush hours, customers arrive at the rate of 10 per hour. The average number of customers that can be served is 12 per hour. Find out the following:

1. Probability that the ticket counter is free.
1. Average number of customers in the queue.

### Solution

Given  

$\lambda = 10/hour$, $\mu = 12/hour$

1. Probability that the charging station is free ($P_o$) = $1 - \cfrac{\lambda}{\mu} = 1 - \cfrac{10}{12} = \cfrac{1}{6}$   

1. Average number of customers in the queue ($L_q$ ) = $\cfrac{\lambda^2}{\mu(\mu-\lambda)} = \cfrac{10^2}{12(12-10)} = \cfrac{25}{6} = 4.16666$


#### Simulation of Example 2

In [6]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)

    def process(self):
        while True:
            Car(name="Car")
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)
        
    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 10)  # lambda = 10
srv_distr = sim.Exponential(60 / 12)  # mu = 12

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()
# chargingstations.requesters().length.print_statistics()
# chargingstations.occupancy.print_statistics()
P0 = 1 - chargingstations.occupancy.mean()
Lq = chargingstations.requesters().length.mean()

print(
    "\n",
    "Probability that the charging station is free: \t",
    P0,
    "\n",
    "Average number of customers in the queue: \t\t",
    Lq,
)


 Probability that the charging station is free: 	 0.16245612432887735 
 Average number of customers in the queue: 		 4.224836568143225


## Example 3

At Bharat charging station, cars arrive according to a Poisson process with an average time of 5 minutes between arrivals. The service time is exponentially distributed with mean time = 2 minutes. On the basis of this information, find out

1. What would be the average queue length?
1. What would be the average number of customers in the queuing system?
1. What is the average time spent by a car in the petrol pump?
1. What is the average waiting time of a car before receiving petrol?

### Solution

Given  

$\lambda = 12/hour$, $\mu = 30/hour$

Average inter arrival time = $\cfrac{1}{\lambda} = 5 minutes = \cfrac{1}{12} = \lambda = 12/hour$
 	 	 	 	 
Average service time =	$\cfrac{1}{\mu} = 2 minutes = \cfrac{1}{30} = \mu = 30/hour$


 Performance Indicator                                                 | Formula 
:----------------------------------------------------------------------|:-----------------------------------------------
 Average queue length ($L_q$ )                                         | $ = \cfrac{12^2}{30(30-12)} = \cfrac{4}{15}$
 Average number of customers, ($L_s$)                                  | $ = \cfrac{12}{30-12} = \cfrac{2}{3}$	
 Average time spent at the charging station ($W_s$)                    | $ = \cfrac{1}{\mu-\lambda} = \cfrac{1}{30-12} = $ 3.33 minutes
 Average waiting time of a car before being charged ($W_q$)            | $ = \cfrac{12}{30(30-12)} = $ 1.33 minutes

In [7]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)

    def process(self):
        while True:
            Car(name="Car")
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 12)  # lambda = 12/hour
srv_distr = sim.Exponential(60 / 30)  # mu     = 30/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,
)


 Average queue length: 					 0.2630923686223894 
 Average number of customers: 				 0.6635214843976908 
 Average time spent at the charging station: 		 3.3150673229150724 
 Average waiting time of a car before being charged: 	 1.3150673229150722


## Example 4

Enexis is considering to operate a single charging station. Management estimates that customers will arrive at the rate of 15 per hour. The charging station can charge a car at the rate of one every 3 minutes.

Assuming Poisson arrivals and exponential service find

1. Average number in the waiting line.
1. Average number in the system.
1. Average waiting time in line.
1. Average waiting time in the system.

### Solution.
Given  

$\lambda = 15/hour$, $\mu = 20/hour$

 Performance Indicator                                                 | Formula 
:----------------------------------------------------------------------|:-----------------------------------------------
 Average queue length ($L_q$ )                                         | $ = \cfrac{15^2}{20(20-15)} = $ 2.25 cars
 Average number of customers, ($L_s$)                                  | $ = \cfrac{15}{20-15} = $ 3 cars	
 Average time spent at the charging station ($W_s$)                    | $ = \cfrac{1}{\mu-\lambda} = \cfrac{1}{20-15} = $ 12 minutes
 Average waiting time of a car before being charged ($W_q$)            | $ = \cfrac{15}{20(20-15)} = $ 0.15 hours = 9 minutes

In [8]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)        

    def process(self):
        while True:
            Car(name="Car")
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)        

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 15)  # lambda = 12/hour
srv_distr = sim.Exponential(60 / 20)  # mu     = 30/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,
)


 Average queue length: 					 2.121157478850578 
 Average number of customers: 				 2.86666466723288 
 Average waiting time of a car before being charged: 	 8.427324111444467 
 Average time spent at the charging station: 		 11.427324111444467


## Example 5

Chhabra Saree Emporium has a single charging station. During the rush hours, cars arrive at the rate of 10 per hour. The average number of customers that can be processed by the charging station is 12 per hour. On the basis of this information, find the following:

1. Probability that the charging station is idle
1. Average number of cars in the queuing system
1. Average time a customer spends in the system
1. Average number of customers in the queue
1. Average time a customer spends in the queue

### Solution.
Given  

$\lambda = 10/hour$, $\mu = 12/hour$

 Performance Indicator                                     | Formula 
:----------------------------------------------------------|:-----------------------------------------------
 Probability of zero unit in the queue ($P_o$)             | $ = 1 - \cfrac{\lambda}{\mu} = 1 - \cfrac{10}{12} = \cfrac{1}{6}$
 Average number of units in the system ($L_s$)             | $ = \cfrac{\lambda}{\mu-\lambda} = \cfrac{10}{12-10} = $ 5 cars
 Average waiting time of an arrival in the system ($W_s$)  | $ = \cfrac{1}{\mu-\lambda} = \cfrac{1}{12-10} =$ 30 minutes
 Average queue length ($L_q$ )                             | $ = \cfrac{\lambda^2}{\mu(\mu-\lambda)} = \cfrac{10^2}{12(12-10)} = \cfrac{25}{6}$ cars
 Average waiting time of an arrival ($W_q$)                | $ = \cfrac{\lambda}{\mu(\mu-\lambda)} = \cfrac{10}{12(12-10)} = $ 25 minutes
 

In [9]:
# Resource method
# https://www.salabim.org/manual/Modelling.html#the-bank-office-example-with-resources

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)        

    def process(self):
        while True:
            Car(name="Car")
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)        

    def process(self):
        self.request(chargingstations)
        self.hold(srv_distr.sample())
        self.release()  # not really required


N_STATION = 1
iat_distr = sim.Exponential(60 / 10)  # lambda = 10/hour
srv_distr = sim.Exponential(60 / 12)  # mu     = 12/hour

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Instantiate the servers, list comprehension but only 1 server
chargingstations = sim.Resource(
    name="Charging Stations", capacity=N_STATION, monitor=True
)
chargingstations.requesters().length_of_stay.monitor(value=True)

# Execute Simulation
app.run(till=50000)

# Print statistics
# chargingstations.print_statistics()

P0 = 1 - chargingstations.occupancy.mean()
Lq = chargingstations.requesters().length.mean()
Ls = Lq + chargingstations.occupancy.mean()
Wq = chargingstations.requesters().length_of_stay.mean()
Ws = Wq + srv_distr.mean()


print(
    "\n",
    "Probability that the charging station is free: \t",
    P0,
    "\n",
    "Average number of customers: \t\t\t\t",
    Ls,    
    "\n",
    "Average time spent at the charging station: \t\t",
    Ws,    
    "\n",
    "Average queue length: \t\t\t\t\t",
    Lq,
    "\n",
    "Average waiting time of a car before being charged: \t",
    Wq,

)


 Probability that the charging station is free: 	 0.17945065092892432 
 Average number of customers: 				 4.441818469732978 
 Average time spent at the charging station: 		 26.878136301727302 
 Average queue length: 					 3.6212691206619025 
 Average waiting time of a car before being charged: 	 21.878136301727302
